# Simulation Chaotic Magnetic Pendulum

This code calculates the positions and velocities of the magnetic pendulum using the positions of the arragment of magnets on the table and the initial conditions of this magnetic pendulum. The solution was obtained using the RK4 method. Besides, this code also analysis the data obtaines throw animations of the movement, phase portraits, Liapunov exponents, and basins of atraction.

The structure of the code follows the nex points:
1. [Create the code to find and animate the trajectories](#section1)
    1. Class of magnetic pendulum
    2. Animation
2. [Run the code and show results](#section2)
    1. Find path
    2. Plot and animate trajectory
3. [Analyse the data](#section3)
    1. Phase Portrait
    2. Difference of paths and Liapunov exponent
    3. Basisns of atraction

In [1]:
from matplotlib import cm  #Color maps
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time

## 1. Create the code to find and animate the trajectories  <a id='section1'></a>

### A. Class of the magnetic pendulum

In [2]:
class pendulum:
    def __init__(self):
        # System dynamics parameters
        self.R = 0.01          #friction constant   
        self.l = 0.54           #Pendulum length
        self.d = 0.03       #vertical distance between pendulum and magnets
        self.h = 0.01       #time lapse between each step
        self.N = 2000       #total number of steps
        self.m = 0.036226       #Mass of the pendulum magnet
        self.mu_P_magn = 2       #Magnitude of the magnetic dipole of the pendulum magnet
        
        # Other parameters
        self.sec = 1/self.h #Number of steps that waste one second.
        self.Table_SIZE = 0.3
        self.NMAGNETS = np.random.randint(low=1,high=8)
        self.lim = self.Table_SIZE/2 + 0.04         #Limit of the plots
        self.lim_vs = (self.Table_SIZE + 0.05)/2   #Limit plot of vector space
        
        # Initial velocities
        self.vx = 0
        self.vy = 0
        
        # Set initial position randomly    
        lm = self.Table_SIZE
        self.x = np.random.rand()*lm -lm/2
        self.y = np.random.rand()*lm -lm/2
              
        # Create table
        self.Mx = []
        self.My = []
        for i in range(self.NMAGNETS):
            self.Mx.append(np.random.rand()*self.Table_SIZE - self.Table_SIZE/2)
            self.My.append(np.random.rand()*self.Table_SIZE - self.Table_SIZE/2)
        
        # Create dipoles
        self.mu = [-1.071875, -2.048, -1.9456, -1.8432, -1.6384, -1.7408, -1.4336]
        self.mu_magn = []   # Magnitud of the magnetic dipoles
        self.mu_dir = []    # Direction of the magnetic dipoles
        self.S = [] 
        self.mu_NUMS=len(self.mu)
        for i in range(self.NMAGNETS): #set the magnetic dipoles to the magnets randomly
            
            num = np.random.randint(self.mu_NUMS)
            mu = self.mu[num]
            
            self.S.append(np.abs(mu))
            self.mu_magn.append(np.abs(mu))
            self.mu_dir.append(np.array([0,0,-1])*np.sign(mu)) #The directions are in the z-axis
        
        self.mu_dir = np.array(self.mu_dir)
        self.sign = np.array([self.mu_dir[i][2]*-1 for i in range(self.NMAGNETS)])
        
        #Finding S values
        self.S = np.array(self.S)
        self.S = 3*(4*np.pi*10**-7 * self.mu_P_magn) / (4*np.pi * self.m) * self.S
    
    def set_lim(self, lim):
        self.lim = lim
    
    def set_positions(self,CI, Mx, My):
        self.x = CI[0]
        self.y = CI[1]
        self.vx = CI[2]
        self.vy = CI[3]
        
        self.Mx = Mx
        self.My = My
        self.NMAGNETS = len(Mx)
        
    def set_code_parameters(self, m, l, d):
        self.m = m     #Mass of the pendulum
        self.l = l     #lenght of the pendulum
        self.d = d     #Distance between table and pendulum equilibrium position
    
    def set_magnetic_dipoles(self, mu_P, mu):
        self.mu_P_magn = np.abs(mu_P)  #Magnitude of the magnetic dipole
        self.mu_magn = np.abs(np.array(mu))
        
        self.mu_dir = [np.array([0,0,-1])*np.sign(mu[i]) for i in range(self.NMAGNETS)] #Direction magnetic dipoles of table magnets
        self.mu_dir = np.array(self.mu_dir)
        self.mu_NUMS=len(self.mu_magn)
        
        #Finding S values
        self.S = np.copy(self.mu_magn)
        self.S = 3*(4*np.pi*10**-7 * self.mu_P_magn) / (4*np.pi) * self.S
    
        self.sign = [np.sign(mu[i]) for i in range(self.NMAGNETS)] #If sign is positive there is atraction; if negative, repulsion
    
    def get_self(self):
        return self
    
    def find_path(self):
        # Finding the next steps for each interval
        self.X = [self.x]    #X positions
        self.Y = [self.y]    #Y positions
        self.Vx = [self.vx]  #X velocities
        self.Vy = [self.vy]  #Y velocities
        self.FB = []         #Arrays of the net magnetic forces
        
        s=self.sec #Number of steps that waste one second.
        A = int(1.2*s) #check point 1
        B = int(0.6*s) #check point 2
        C = int(0.2*s) #check point 3
        Rc = (0.01)**2 #squared Critical distance to stop the calculations. The magnitude is in meters^2
        
        #Finfing path
        for t in range(self.N):
            next_val=self.next_value(self.X[t],self.Y[t],self.Vx[t],self.Vy[t])
            
            self.FB.append(self.FF2)
            self.X.append(next_val[0])
            self.Y.append(next_val[1])
            self.Vx.append(next_val[2])
            self.Vy.append(next_val[3])
            
            if t>5*s:  #This conditional returns true when the time is higher than 5 seconds
                if ((self.X[t-A]-self.X[t])**2 + (self.Y[t-A]-self.Y[t])**2 < Rc):
                    if ((self.X[t-B]-self.X[t])**2 + (self.Y[t-B]-self.Y[t])**2 < Rc):
                        if ((self.X[t-C]-self.X[t])**2 + (self.Y[t-C]-self.Y[t])**2 < Rc):
                            break
            
        # Find the last net magnetic force
        aux = self.next_value(self.X[-1],self.Y[-1],self.Vx[-1],self.Vy[-1])
        self.FB.append(self.FF1)
        
        # Converting list to arrays
        self.X = np.array(self.X)
        self.Y = np.array(self.Y)
        self.Vx = np.array(self.Vy)
        self.Vy = np.array(self.Vx)
        
        return self.X, self.Y, self.Vx, self.Vy
    
    def plot_path(self):
        # Plot trajectory
        plt.plot(self.X,self.Y,'-b',linewidth=0.5)
        
        # Initial and final position
        plt.plot(self.X[0],self.Y[0],'Dg',label=f'Initial position ({np.round(self.X[0],3)},{np.round(self.Y[0],3)})', markersize=5)
        plt.plot(self.X[-1],self.Y[-1],'Dc',label='Final position', markersize=5)
        
        # Put legends, title, labales, and ticks
        plt.legend(fontsize=12)
        plt.title('Trajectory of the magnet',fontsize=15)
        plt.xlabel('x-axis [m]',fontsize=12)
        plt.ylabel('y-axis [m]',fontsize=12)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        
    def plot_table(self):
        
        # Plot magnets
        plt.figure(figsize=(6,6))
        for i in range(self.NMAGNETS):
            plt.plot(self.Mx[i],self.My[i],'or')
        plt.plot(self.Mx[0],self.My[0],'or',label='Magnets')
        
        # Plot Table
        self.ap = self.Table_SIZE/2 +0.01
        plt.plot([-self.ap,-self.ap,self.ap,self.ap,-self.ap],[-self.ap,self.ap,self.ap,-self.ap,-self.ap],'-b',label='Table')
        
        # Put legends, title, labales, and limits
        plt.legend()
        plt.title('Positions of magnets')
        plt.xlabel('x-axis [m]')
        plt.ylabel('y-axis [m]')
        plt.xlim(-self.lim,self.lim)
        plt.ylim(-self.lim,self.lim) 

    def plot_vector_space(self,res=20,a=0):
        
        # Put limits of plot
        if a==0:
            a=self.lim_vs #Predefined value
        elif a > self.l:
            a = self.l/2 #Ensure that the points are ploted in the zone inside the spheric pendulum
        
        # Create data of vector tails
        t = np.linspace(-a,a,res)
        NUM = len(t)
        X, Y = np.meshgrid(t,t)
        Z = -np.sqrt(self.l**2 - X**2 - Y**2)+self.l+self.d
        
        # Finding net magnetic force
        Fx = np.zeros((NUM,NUM))
        Fy = np.zeros((NUM,NUM))
        Fz = np.zeros((NUM,NUM))
        
        # Unit vector of the magnetic dipole m1 (from Q to P)
        M1x = X/self.l                 #X component 
        M1y = Y/self.l                 #Y component 
        M1z = (Z-self.l-self.d)/self.l #Z component 
        
        # Iterating over table magnets
        for i in range(self.NMAGNETS):
            
            #Squared distance between magnet i and pendulum magnet
            R = (self.Mx[i]-X)**2 + (self.My[i]-Y)**2 + Z**2  
            
            #Table magnet constants
            m2 = self.mu_dir[i]
            
            #Unit vector from m2 (on table) to m1 (in pendulum)
            Ur_x = (X-self.Mx[i])/np.sqrt(R) #X component 
            Ur_y = (Y-self.My[i])/np.sqrt(R) #Y component 
            Ur_z = Z/np.sqrt(R)              #Z component 
            
            #Dot products
            M12 = M1x*m2[0] + M1y*m2[1] + M1z*m2[2]     #Dot products between m1 and m2
            M1r = M1x*Ur_x + M1y*Ur_y + M1z*Ur_z        #Dot products between m1 and ur
            M2r = m2[0]*Ur_x + m2[1]*Ur_y + m2[2]*Ur_z  #Dot products between m2 and ur
            
            #Sum contributions of all magnets
            Fx = Fx + self.S[i] / R**2 * ( (M12)*Ur_x + (M2r)*M1x + (M1r)*m2[0] - 5*(M1r)*(M2r)*Ur_x )
            Fy = Fy + self.S[i] / R**2 * ( (M12)*Ur_y + (M2r)*M1y + (M1r)*m2[1] - 5*(M1r)*(M2r)*Ur_y )
            Fz = Fz + self.S[i] / R**2 * ( (M12)*Ur_z + (M2r)*M1z + (M1r)*m2[0] - 5*(M1r)*(M2r)*Ur_z )
        
        # Magnitude of the projection of net magnetic force on plane xy
        F_xy_mag = np.sqrt(Fx**2 + Fy**2)
        
        # Finding tension Force
        
        # Unit vector of tension (from P to Q) 
        UT_x = -X/self.l                #X component 
        UT_y = -Y/self.l                #Y component 
        UT_z = (self.l+self.d-Z)/self.l #Z component 
        
        # Dot product
        FbUT = Fx*UT_x + Fy*UT_y + Fz*UT_z  #Dot product between net Fb and UT
        
        # Finding Tension
        Tx = (self.m*9.8*(self.l+self.d-Z)/self.l - FbUT)*UT_x
        Ty = (self.m*9.8*(self.l+self.d-Z)/self.l - FbUT)*UT_y
        Tz = (self.m*9.8*(self.l+self.d-Z)/self.l - FbUT)*UT_z
        
        # Magnitude of the projection of tension force on plane xy
        T_xy_mag = np.sqrt(Tx**2 + Ty**2)
        
        # Find force field (refusing friction force)
        Field_x = Fx+Tx
        Field_y = Fy+Ty
        Field_z = Fz+Tz
        
        # Magnitude of the projection of force field on plane xy
        Field_xy_mag = np.sqrt(Field_x**2 + Field_y**2)
        
        # Plotting vectors
        
        # Field Forces
        plt.quiver(X,Y,Field_x/Field_xy_mag, Field_y/Field_xy_mag, width=0.0025, angles='xy', scale_units='width', scale=36)
        
        #Magnetic net forces
        #plt.quiver(X,Y,Fx/F_xy_mag,Fy/F_xy_mag, linewidths=4, width=0.0025, angles='xy', scale_units='width', scale=39)                
        
        #Tensions
        #plt.quiver(X,Y,Tx/T_xy_mag, Ty/T_xy_mag, linewidths=4, width=0.0025, angles='xy', scale_units='width', scale=37)

    
    def plot_potential(self,res=20, a=0):
        
        # Put limits of plot
        if a==0:
            a = self.lim_vs #predifined value
        elif a > self.l:
            a = self.l/2 #Ensure that the points are ploted in the zone inside the spheric pendulum
        
        # Create data of the plot
        t = np.linspace(-a,a,res)
        NUM = len(t)
        X, Y = np.meshgrid(t,t)
        Z = -np.sqrt(self.l**2 - X**2 - Y**2)+self.l+self.d

        # Finding magnetic field B
        Bx = np.zeros((NUM,NUM))
        By = np.zeros((NUM,NUM))
        Bz = np.zeros((NUM,NUM))
        
        # Unit vector of the magnetic dipole of m1 (vector from Q to P)
        M1x = X/self.l                 #X component
        M1y = Y/self.l                 #Y component 
        M1z = (Z-self.l-self.d)/self.l #Z component 
        
        # Iterating over each table magnet
        for i in range(self.NMAGNETS):
            
            #Squared distance between magnet i and the pendulum magnet
            R = (self.Mx[i]-X)**2 + (self.My[i]-Y)**2 + Z**2  
            
            #Table magnet constants
            m2 = self.mu_dir[i]
            Cm = -(4*np.pi*10**-7 * self.mu_magn[i]) / (4*np.pi)
            
            #Unit vector from m2 (on table) to m1 (in pendulum)
            Ur_x = (X-self.Mx[i])/np.sqrt(R) #X component
            Ur_y = (Y-self.My[i])/np.sqrt(R) #Y component 
            Ur_z = Z/np.sqrt(R)              #Z component 
            
            #Dot products
            M2r = m2[0]*Ur_x + m2[1]*Ur_y + m2[2]*Ur_z  #Dot products between m2 and ur
            
            #Sum contributions of all magnets
            Bx = Bx + Cm/R**(3/2) * (m2[0] - 3*M2r*Ur_x)
            By = By + Cm/R**(3/2) * (m2[1] - 3*M2r*Ur_y)
            Bz = Bz + Cm/R**(3/2) * (m2[2] - 3*M2r*Ur_z)
        
        # Finding magnetic potential
        
        # Unit vector from Q to P (direction of m1)
        Um1_x = X/self.l                 #X component  
        Um1_y = Y/self.l                 #Y component 
        Um1_z = (Z-self.l-self.d)/self.l #Z component 
        
        # Pendulum magnet constants
        m1 = self.mu_P_magn
        
        # Magnetic potential
        Um = -m1 * (Um1_x*Bx + Um1_y*By + Um1_z*Bz)
        
        # Finding gravitational potential
        Ug = self.m*9.8*(Z-self.d)
        
        # Plot potential
        fig = plt.figure(figsize=(5,4))
        ax = plt.axes(projection='3d')
        surf = ax.plot_surface(X,Y,Ug+Um, cmap=cm.jet, edgecolor='none')
        fig.colorbar(surf,shrink=0.5, aspect=8, pad = 0.15)
        
        plt.title(f'Potential Energy', fontsize=15)
        plt.xlabel(r'X [m]',fontsize=12)
        plt.ylabel(r'Y [m]',fontsize=12)
        ax.set_zlabel(r'Ug+Um [J]',fontsize=12)
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=9)
        for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(9)
            
    def Fk(self,x,y,vx,vy):
        """
        Calculates the velocity and acelerations based on the initial conditions of the magnets 
        @params
        mx, my:
            arrays of doubles that contains the position of the magnets
        x,y,vx,vy:
            doubles of the position and velocity of the pendulum magnet

        @returns
        vx, vy ,ax, ay:
            doubles that contains the velocity and aceleration based on the given inputs
        """ 
        
        # z component of pendulum magnet
        z = -np.sqrt(self.l**2-x**2-y**2)+self.l+self.d       
        
        # Magnetic dipole vector m1 of pendulum magnet.     
        
        # Vector from Q to P
        rqp = np.array([x,y,z-self.l-self.d])
        
        # Magnetic dipole
        m1 = rqp/self.l
        
        # Finidng the sum of the magnetic forces
        Fb = np.array([0,0,0])
        for i in range(self.NMAGNETS):
            #Squared distance between pendulum and table magnet
            r = (self.Mx[i]-x)**2 + (self.My[i]-y)**2 + z**2
            
            #Magnetic dipole vector of the table magnet
            m2 = self.mu_dir[i]
            
            #Unit vector of relative position (from m2 to m1)
            ur = np.array( [x-self.Mx[i], y-self.My[i], z] )/np.sqrt(r) 
            
            #Sum of the forces
            Fb = Fb + self.S[i] * ( (m1@m2)*ur + (m2@ur)*m1 + (m1@ur)*m2 - 5*(m1@ur)*(m2@ur)*ur ) / r**2
        
        # Save the data of the net magnetic force.
        self.FF1 = Fb
        
        # Finding tension
        
        # Unit vector from P to Q
        uT = -rqp/self.l 
        
        # Tension
        T = (self.m*9.8*(self.l+self.d-z)/self.l - Fb@uT)*uT
        
        # Acelerations
        ax = (Fb[0] - self.R*vx + T[0])/self.m
        ay = (Fb[1] - self.R*vy + T[1])/self.m
               
        return np.array([vx, vy, ax, ay])

    
    def next_value(self, x,y,vx,vy):
        """
        Calculates the next positions and velocities after a time lapse of self.h
        Find the 4 k-values of the Runge-Kutta method (RK4)
        @params
        x,y,vx,vy:
            doubles of the position and velocity of the pendulum magnet

        @returns
        xf,yf,vxf,vyf
            doubles that contains the final positions and velocities of the pendulum magnet
        """
        
        #K1 and calculations
        k1 = self.h*self.Fk(x,y,vx,vy)
        x1 = x + k1[0]/2
        y1 = y + k1[1]/2
        vx1 = vx + k1[2]/2
        vy1 = vy + k1[3]/2
        
        #Save magnetic Force
        self.FF2 = self.FF1

        #K2 and calculations
        k2 = self.h*self.Fk(x1,y1,vx1,vy1)
        x2 = x + k2[0]/2
        y2 = y + k2[1]/2
        vx2 = vx + k2[2]/2
        vy2 = vy + k2[3]/2

        #K3 and calculations
        k3 = self.h*self.Fk(x2,y2,vx2,vy2)
        x3 = x + k2[0]
        y3 = y + k2[1]
        vx3 = vx + k2[2]
        vy3 = vy + k2[3]

        #K4 and calculating final positions
        k4 = self.h*self.Fk(x3,y3,vx3,vy3)
        xf = x + 1/6*(k1[0] + 2*k2[0] + 2*k3[0] + k4[0])
        yf = y + 1/6*(k1[1] + 2*k2[1] + 2*k3[1] + k4[1])
        vxf = vx + 1/6*(k1[2] + 2*k2[2] + 2*k3[2] + k4[2])
        vyf = vy + 1/6*(k1[3] + 2*k2[3] + 2*k3[3] + k4[3])

        return np.array([xf, yf, vxf, vyf])
    
print('Done')

Done


### B. Animation

In [3]:
def animate_path(self):
    
    #local function for update each frame
    def update(j):
        
        # Clear the before plot
        ax.clear()
        
        # Plot the trajectories until the interval j
        ax.plot(self.X[:j],self.Y[:j],'-b',linewidth=0.5)
        ax.plot(self.X[j],self.Y[j],'o',markersize=5,color='b')
    
        # Plot the magnets
        for i in range(self.NMAGNETS):
            ax.plot(self.Mx[i],self.My[i],'or',markersize=10)
            ax.text(self.Mx[i]-0.02,self.My[i]-0.04,f'{np.round(self.mu_magn[i]*self.sign[i],2)}', fontsize=8)
        
        # Plot initial and final position
        ax.plot(self.X[0],self.Y[0],'Dg',label=f'Initial position ({np.round(self.X[0],3)},{np.round(self.Y[0],3)})', markersize=5)
        ax.plot(self.X[-1],self.Y[-1],'Dc',label='Final position', markersize=5)
        
        # Finding net magnetic Force
        FBx = self.FB[j][0]
        FBy = self.FB[j][1]
        FB_xy_mag = np.sqrt(FBx**2+FBy**2)
        
        # Plot net magnetic force vector
        ax.quiver(self.X[j],self.Y[j],0.25*FBx/FB_xy_mag,0.25*FBy/FB_xy_mag,color='red', label='Net Fb',angles='xy', scale_units='xy', scale=2)
        
        # Put on legend the Iteration and time of the frame
        ax.plot(0,0,'o',markersize=0, label=f'Iteration {j}')
        ax.plot(0,0,'o',markersize=0, label=f'Time {np.round(j*self.h,2)}s')
        
        # Put grids, legends, and limits
        plt.grid()
        plt.legend()
        plt.xlim(-self.lim,self.lim)
        plt.ylim(-self.lim,self.lim) 
    
    # Create figure and axis
    fig = plt.figure(figsize=(6,6))
    ax = fig.gca()
    
    # Put labels, title, grids, and legend
    plt.xlabel('X-axis [m]')
    plt.ylabel('Y-axis [m]')
    plt.title('Pendulum magnet trajectory')
    plt.grid()
    plt.legend()
    
    # Approximately Time per each plot
    f = 1/(0.25) 
    
    # Number of intervals that animation show per frame. Higher 'steps' lower the time of animation. steps < N
    steps=int(1/(self.h*f)) 
    
    # Number of intervals
    N = len(self.X)
    
    # Animate the movement
    self.ani=animation.FuncAnimation(fig,update,range(10,N,steps), repeat=False) 
    
    # Show animation
    plt.show()
print('Done')

Done


In [5]:
%matplotlib auto

Using matplotlib backend: TkAgg


## 2. Run the code and show results <a id='section2'></a>

### A. Find path

In [6]:
# Create pendulums
NUM = 12 #Número de pruebas
pends = [pendulum() for i in range(NUM)]

# Initial conditions
CI = [[0.11, -0.11,0,0],[0.11-0.001, -0.11+0.001,0,0],    #Sensibility case 1
      [0.14, 0,0,0],[0.14+0.001,0-0.001,0,0],            #case 2  
      [0, 0.16,0,0],[0+0.001, 0.16+0.001,0,0],            #case 3
      [0.148, 0.115,0,0],[0.148+0.001, 0.115+0.001,0,0],   #case 4
      [-0.0318, 0.049,0,0],[-0.0318-0.001, 0.049-0.001, 0,0], #case 5
      [-0.0248, -0.002,0,0],[-0.0248+0.001, -0.002+0.001,0,0]] #case 6

# Create table
radius = 0.10
angles = [30, 90, 150, 210, 270, 330]
angles = np.deg2rad(np.array(angles))
Mx = radius*np.cos(angles)
My = radius*np.sin(angles)

Mx, My = list(Mx), list(My)
Mx.append(0)
My.append(0)

# Code parameters
m = 0.036226  #mass
l = 0.54      #pendulum length
d = 0.03      #distance between table and pendulum equilibrium position
   
# Magnetic Dipoles
mu_P = 1.84   #magnetic dipole magnitude (pendulum)
mu = [2.5]*6 #magnetic dipoles magnitudes (table). Negative for repulsion, positive for atraction
mu.append(-2.5)

for i in range(NUM):
    pends[i].set_positions(CI[i],Mx,My)
    pends[i].set_code_parameters(m,l,d)
    pends[i].set_magnetic_dipoles(mu_P,mu)
    inicio = time.time()
    X, Y, Vx, Vy = pends[i].find_path()
    fin = time.time()
    print('Time',fin-inicio)  

# Get self
selfs = [pends[i].get_self() for i in range(NUM)]

Time 0.6123785972595215
Time 0.409848690032959
Time 0.4884779453277588
Time 0.3891303539276123
Time 0.3949451446533203
Time 0.5855345726013184
Time 0.43341732025146484
Time 0.4353504180908203
Time 0.4996936321258545
Time 0.44542503356933594
Time 0.5810725688934326
Time 0.6382930278778076


### B. Plot and animate trajectory

In [7]:
# Plot the potential
k=0
pends[k].plot_potential(res=100) #res=resolution (optional), a=limits of plot (optional)
plt.savefig('Hpos_potential.png',dpi=300)

In [8]:
# Plot magnets
pends[0].plot_table()
Mx = selfs[0].Mx
My = selfs[0].My

# Plot magnets
for i in range(len(Mx)):
    plt.plot(Mx[i],My[i],'or')
    plt.text(Mx[i]+0.005,My[i]+0.005,f'{i}')

# Put limits, labels, legend and title
lim = selfs[0].Table_SIZE/2 + 0.04
plt.xlim(-lim,lim)
plt.ylim(-lim,lim)
plt.xlabel('X-axis [m]')
plt.ylabel('Y-axis [m]')
plt.legend(loc='upper right')
plt.title('Table Magnets')

Text(0.5, 1.0, 'Table Magnets')

In [9]:
# Plot the table, trajectory, and vector field.

for i in range(10,12):
    # Plot table and path
    pends[i].plot_table()              
    pends[i].plot_path()
    plt.savefig(f'Hpos_path_{i+1}.png',dpi=300)

    # Plot vector space
    pends[i].plot_vector_space(res=30) #res: resolution. Increase to obtain more vectors.
    plt.savefig(f'Hpos_vector_space_{i+1}.png',dpi=300)

In [10]:
# Animate the trajectory
k=0
pends[k].set_lim(0.25) #Adjust the xlims and ylims of the plot. Increase to see small the set up.
animate_path(selfs[k])

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


## 3. Analyse the data <a id='section3'></a>

### A. Phase Portrait

In [12]:
#Phase portrait 3D (x,y,v)

for i in range(10,12):
    # Get positions and velocity
    X=selfs[i].X
    Y=selfs[i].Y
    Vx=selfs[i].Vx
    Vy=selfs[i].Vy
    V = np.sqrt(Vx**2+Vy**2)*np.sign(Vy) #With sign

    # Create projection 3d
    plt.figure()
    ax = plt.axes(projection='3d')

    # Plot phae portrait
    plt.plot(X,Y,V,'-r',linewidth=0.5)

    # Put tilte and labels
    plt.title(f'Phase portrait path {i+1}', fontsize=15)
    plt.xlabel(r'X [m]',fontsize=12)
    plt.ylabel(r'Y [m]',fontsize=12)
    ax.set_zlabel(r'V sign(Vx) [m/s]',fontsize=12)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(9)
    plt.savefig(f'Hpos_phase_portrait_{i+1}.png',dpi=300)
    plt.show()

    # Time series
    plt.figure()

    # Create time array
    t=np.linspace(0,selfs[i].h*selfs[i].N,len(X))

    # Plot time series
    plt.plot(t,X,'-b',linewidth=0.5)

    # Put the title and labels
    plt.title(f'Time series of X path {i+1}',fontsize=15)
    plt.xlabel('Time [s]',fontsize=12)
    plt.ylabel('X [m]',fontsize=12)
    plt.savefig(f'Hpos_time_series_of_X_{i+1}.png',dpi=300)
    plt.show()

In [13]:
# Superposition
plt.figure()
ax = plt.axes(projection='3d')
colors=['blue','red']*(NUM//2)
#colors=['blue','red','green','orange']*3
for i in range(10,12):
    # Get positions and velocity
    X=selfs[i].X
    Y=selfs[i].Y
    Vx=selfs[i].Vx
    Vy=selfs[i].Vy
    V = np.sqrt(Vx**2+Vy**2)*np.sign(Vy) #With sign
    
    # Plot phae portrait
    plt.plot(X,Y,V,color=colors[i],linewidth=0.8,label=f'Path {i+1}',alpha=0.9)

# Put tilte and labels
plt.title('Phase portrait 3D', fontsize=15)
plt.xlabel(r'X [m]',fontsize=12)
plt.ylabel(r'Y [m]',fontsize=12)
plt.legend()
ax.set_zlabel(r'V sign(Vx) [m/s]',fontsize=12)
plt.xticks(fontsize=9)
plt.yticks(fontsize=9)
for t in ax.zaxis.get_major_ticks(): t.label.set_fontsize(9)
plt.show()
plt.savefig(f'Hex_pos_phase_portrait_{i}{i+1}.png',dpi=300)

### B. Difference of paths and Liapunov exponent

In [14]:
# Differents between paths
case=6

X1, Y1 = selfs[2*case-2].X, selfs[2*case-2].Y 
X2, Y2 = selfs[2*case-1].X, selfs[2*case-1].Y

# Distance between paths
nn = min([len(X1),len(X2)])
D=np.sqrt((X1[:nn]-X2[:nn])**2 + ((Y1[:nn]-Y2[:nn])**2))

# Time array
t=np.linspace(0,selfs[0].h * selfs[0].N ,len(D))

plt.figure()

# Plot difference vs time
#plt.plot(t,D/max(D), label='Distance')
plt.plot(t, np.log(D/D[0]), '-b',label='Log distance')

# Put labels, title, legend, ticks and grid
plt.ylabel('Distance between trajectories [m]', fontsize = 12)
plt.xlabel('Time [s]', fontsize = 12)
plt.title('Chaotic behaviour, changing 1mm the initial positions', fontsize = 15)
plt.legend(fontsize=10, ncol=1)
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.grid(True)

# Save image
plt.savefig(f'Hpos_diference_trajectories_{2*case-1}{2*case}.png',dpi=300)

# Show image
plt.show()

In [91]:
len(t)
t[37] - t[4]

0.8788282290279628

In [15]:
starts = [71, 17, 0, 0, 0, 4]
ends =   [88, 50, 0, 0, 0, 37]

start = starts[case-1]
end = ends[case-1]

def linear(x, m, b):
    return x*m + b

t_set = t[start:end+1]
D_set = D[start:end+1]
params, cov = curve_fit(linear, t_set, np.log(D_set/D_set[0]))
var = np.sqrt(np.diag(cov))

# Plot difference vs time
plt.figure()
plt.plot(t_set, np.log(D_set/D_set[0]),color='blue', label='Log distance')
plt.plot(t_set, linear(t_set, params[0], params[1]), color='orange', label=f'Linear Fit: log(D)={np.round(params[0],2)} t {np.round(params[1],2)}')

# Put labels, title, legend, ticks and grid
plt.ylabel('Log(D(t) / D0) [dimensionless]', fontsize = 12)
plt.xlabel('Time [s]', fontsize = 12)
plt.title('Chaotic behaviour, changing 1mm initial positions', fontsize = 15)
plt.legend(fontsize=10, ncol=1)
plt.xticks(size = 10)
plt.yticks(size = 10)

xtext = 5/7*(t_set[-1]-t_set[0]) + t_set[0]
log_set = np.log(D_set/D_set[0])
ytext = 2/6*(max(log_set)-min(log_set)) + min(log_set)

plt.text(xtext, ytext, f'lambda = {np.round(params[0],3)}', fontsize=10)
plt.grid(True)
plt.savefig(f'Hpos_Lyapunov_{2*case-1}{2*case}.png',dpi=300)
plt.show()

In [16]:
plt.figure()
t_Ly = np.linspace(t[start], t[end-1],200)
y_Ly = D[start]*np.e**(params[0]*(t_Ly)+params[1])

plt.plot(t_Ly, y_Ly, '-',label=f'Fit D(t) = {np.round(D[start],4)}e^({np.round(params[0],2)} t - {-np.round(params[1],2)})',linewidth=1.5, color='red', alpha=0.6)

#start2 = 121
#end2 = 166
#t_Ly2 = np.linspace(t[start2], t[end2+2],200)
#y_Ly2 = D[start2]*np.e**(params2[0]*(t_Ly2)+params2[1])
#plt.plot(t_Ly2, y_Ly2, '-',label=f'Fit 2 D(t) = {np.round(D[start2],4)}e^({np.round(params2[0],2)} t - {-np.round(params2[1],2)})',linewidth=1.5, color='orange', alpha=0.6)
plt.plot(t,D,'-b',label='Distance', linewidth=1.5, alpha=0.6)

plt.ylabel('D(t) [m]', fontsize = 12)
plt.xlabel('Time [s]', fontsize = 12)
plt.ylim(-0.0055, 0.24)
plt.title('Chaotic behaviour, changing 1mm initial positions', fontsize = 15)
plt.legend(fontsize=10, ncol=1, loc='upper left')
plt.xticks(size = 10)
plt.yticks(size = 10)
plt.savefig(f'Hpos_Exponential_divergence_{2*case-1}{2*case}.png',dpi=300)
plt.show()

### C. Basisns of atraction

In [ ]:
# Set initial parameters and conditions

# Resolution of tha basin of atraction
N=20

# Get data of initial conditions
t=np.linspace(-0.2,0.2,N)
mx, my = np.meshgrid(t,t)
Mat = np.zeros((N,N,2))

# Positions of magnets and magnetic dipoles
thett = np.deg2rad(30)
Mx = [0.1*np.cos(-thett), 0.1*np.cos(3*thett), 0.1*np.cos(7*thett)]         #Positions table magnets
My = [0.1*np.sin(-thett), 0.1*np.sin(3*thett), 0.1*np.sin(7*thett)]  
m = 0.036226                   #mass
mu_P = 1.84                    #magnetic dipole magnitude (pendulum)
mu = [1.94, 1.94, 1.94]  #magnetic dipoles in atraction form (table)

In [ ]:
#Find the data of final positions

aux=-1
inicio = time.time()

# Run over each x initial condition
for i in range(0,len(mx)):
    
    # Print the advances of calculations and save the progress
    val = i/(N-1)*15
    if int(val) != aux:
        print('[','#'*int(val), ']',f'{int(val)}/15',time.strftime("%Y-%m-%d %H:%M:%S",time.gmtime()))
        aux=int(val)
        np.savetxt('Mat_adv_x.csv', Mat[:,:,0], delimiter=',')
        np.savetxt('Mat_adv_y.csv', Mat[:,:,0], delimiter=',')
    
    # Run over each y initial condition
    for j in range(0,len(my)):
        
        # Create pendulum
        pend = pendulum()
        CI = [mx[i][j],my[i][j],0,0]  #[x,y,vx,vy] initial values.
        
        # Set initial conditions
        pend.set_positions(CI,Mx,My)
        pend.set_magnetic_dipoles(mu_P,mu)
        
        # Find path
        X, Y, Vx, Vy = pend.find_path()
        
        # Get Final positions
        Mat[i][j][0] = X[-1]
        Mat[i][j][1] = Y[-1]

# Save final data
np.savetxt('Mat_op_x.csv', Mat[:,:,0], delimiter=',')
np.savetxt('Mat_op_y.csv', Mat[:,:,1], delimiter=',')

# Show total time
fin = time.time()
print('Total time ',fin-inicio,'s')

[  ] 0/15 2022-04-18 01:15:46
[ # ] 1/15 2022-04-18 01:16:05
[ ## ] 2/15 2022-04-18 01:16:16
[ ### ] 3/15 2022-04-18 01:16:26
[ #### ] 4/15 2022-04-18 01:16:44
[ ##### ] 5/15 2022-04-18 01:16:52
[ ###### ] 6/15 2022-04-18 01:16:59
[ ####### ] 7/15 2022-04-18 01:17:07
[ ######## ] 8/15 2022-04-18 01:17:20
[ ######### ] 9/15 2022-04-18 01:17:30


KeyboardInterrupt: ignored

In [ ]:
#Save the data of this specific run
selfMat = pend.get_self()
np.savetxt(f'Mat_op_x h={selfMat.h} N={selfMat.N}.csv', Mat[:,:,0], delimiter=',')
np.savetxt(f'Mat_op_y h={selfMat.h} N={selfMat.N}.csv', Mat[:,:,1], delimiter=',')

In [ ]:
# Plot image of basin of atraction

im = np.zeros((N,N,3))

# Colour each pixel (i,j)
for i in range(N):
    for j in range (N):
        
        # Find the distances to each magnet (or atractor)
        d=[]
        for k in range(len(Mx)):
            d.append( np.sqrt((Mat[i][j][0] - Mx[k])**2+(Mat[i][j][1]-My[k])**2) )
        
        # Set the color based on the minimum distance respecting to each magnet
        index = d.index(min(d))
        im[N-1-i,j, index] = 1

# Show figure
plt.figure()
plt.imshow(im)
plt.savefig('Basin of atraction.png',dpi=300)
plt.show()

In [ ]:
#Plot magnets

pend.plot_table()
colors = ['or','og','ob']
for i in range(len(Mx)):
    plt.plot(Mx[i],My[i],colors[i])
    plt.text(Mx[i]+0.005,My[i]+0.005,f'{i+1}')
    plt.text(Mx[i]-0.02,My[i]-0.02,f'{mu[i]}[Am^2]')

# Put the limits, labels, legend, and title
plt.xlim(-0.2,0.2)
plt.ylim(-0.2,0.2)
plt.xlabel('X-axis [m]')
plt.ylabel('Y-axis [m]')
plt.title('Table Magnets')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Read data if needed
dat = np.genfromtxt('Mat_op_x.csv',delimiter=',')